In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def get_labels_and_count():
  count=0
  d_rels={}
  with open("/content/drive/MyDrive/nlp_assignments/assignment2/aassignment2/dataset/nyt29/relations.txt") as rels:
    for line in rels:
      d_rels[line.strip('\n')]=torch.tensor(count)
      count+=1
    d_rels['none']=torch.tensor(count)
    count+=1
    for key in d_rels:
      d_rels[key]=F.one_hot(d_rels[key], num_classes=count)
    return d_rels,count

def get_entities_and_anns_wo_manipulation(annotation):
  annos=annotation.split('|')
  indices=[]
  entities=[]
  for ann in annos:
    a=ann.split()
    indice=[]
    entity=[]
    count=0
    for item in a:
      if item.isnumeric():
        count+=1
        entity.append(int(item))
        if count==2:
          count=0
          indice.append(entity)
          if not entity in entities:
            entities.append(entity)
          entity=[]
      elif item in d_rels:
        num=d_rels[item]
        indice.append(num)
      else:
        print("ERROR: UNEXPECTED ENTRY FOUND IN TUPLE FILE")
    indices.append(indice)
  datum=[]
  for i in entities:
    for j in entities:
      if i==j:
        continue;
      data=[]
      data.append(i)
      data.append(j)
      res=torch.zeros(l_count)
      for src,dest,label in indices:
        if (i==src and j==dest):
          res = (res+label).float()
      if (torch.sum(res)==torch.tensor(0.)): 
        res=d_rels['none'].float()
      data.append(res)
      datum.append(data)
  return datum



d_rels,l_count=get_labels_and_count()
sentences=[]
with open('/content/drive/MyDrive/nlp_assignments/assignment2/aassignment2/dataset/nyt29/train.sent') as s_file:
  sentences=s_file.read().splitlines()
with open('/content/drive/MyDrive/nlp_assignments/assignment2/aassignment2/dataset/nyt29/train.pointer') as l_file:
  anns=l_file.read().splitlines()
dataset=[]
for sentence,annotation in zip(sentences,anns):
  datum=get_entities_and_anns_wo_manipulation(annotation)
  for data in datum:
    data.append(sentence)
    dataset.append(data)
train_df=pd.DataFrame(dataset, columns=['source_entity','destination_entity','labels','text'])
print(len(dataset))
print(train_df['labels'].size)
train_df[0:5]


146588
146588


,source_entity,destination_entity,labels,text
0,"[12, 12]","[9, 9]","[tensor(1.), tensor(0.), tensor(0.), tensor(0....","then terrorism struck again , this time in the..."
1,"[9, 9]","[12, 12]","[tensor(0.), tensor(1.), tensor(1.), tensor(0....","then terrorism struck again , this time in the..."
2,"[16, 17]","[32, 32]","[tensor(0.), tensor(0.), tensor(0.), tensor(1....",a12 new york\/region b1-7 enclave for middle c...
3,"[16, 17]","[19, 21]","[tensor(0.), tensor(0.), tensor(0.), tensor(0....",a12 new york\/region b1-7 enclave for middle c...
4,"[32, 32]","[16, 17]","[tensor(0.), tensor(0.), tensor(0.), tensor(0....",a12 new york\/region b1-7 enclave for middle c...


In [4]:

sentences=[]
with open('/content/drive/MyDrive/nlp_assignments/assignment2/aassignment2/dataset/nyt29/dev.sent') as s_file:
  sentences=s_file.read().splitlines()
with open('/content/drive/MyDrive/nlp_assignments/assignment2/aassignment2/dataset/nyt29/dev.pointer') as l_file:
  anns=l_file.read().splitlines()
dataset=[]
for sentence,annotation in zip(sentences,anns):
  datum=get_entities_and_anns_wo_manipulation(annotation)
  for data in datum:
    data.append(sentence)
    dataset.append(data)
val_df=pd.DataFrame(dataset, columns=['source_entity','destination_entity','labels','text'])
print(len(dataset))
print(val_df['labels'].size)
val_df[0:5]

16344
16344


,source_entity,destination_entity,labels,text
0,"[50, 51]","[17, 17]","[tensor(0.), tensor(0.), tensor(0.), tensor(0....",mr. scruggs -- who is arguing the case with hi...
1,"[50, 51]","[69, 70]","[tensor(0.), tensor(0.), tensor(0.), tensor(0....",mr. scruggs -- who is arguing the case with hi...
2,"[50, 51]","[72, 74]","[tensor(0.), tensor(0.), tensor(0.), tensor(0....",mr. scruggs -- who is arguing the case with hi...
3,"[50, 51]","[62, 62]","[tensor(0.), tensor(0.), tensor(0.), tensor(0....",mr. scruggs -- who is arguing the case with hi...
4,"[17, 17]","[50, 51]","[tensor(0.), tensor(0.), tensor(0.), tensor(0....",mr. scruggs -- who is arguing the case with hi...


In [5]:
! pip install tokenizers
! pip install transformers

In [6]:
from transformers import BertTokenizer, BertModel,DistilBertTokenizer,DistilBertModel
encoder = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
special_tokens_dict = { 'additional_special_tokens':['SSE','SEE','DSE','DEE']}
num_added_toks = encoder.add_special_tokens(special_tokens_dict)
text=train_df['text'][0]
src_entity=train_df['source_entity'][0]
dst_entity=train_df['destination_entity'][0]
splt=text.split(' ')
print(text,src_entity,dst_entity)
if (src_entity[1]<dst_entity[0]):
  new_text=splt[0:src_entity[0]] + ['SSE'] + splt[src_entity[0]:src_entity[1]+1] + ['SEE'] + splt[src_entity[1]+1:dst_entity[0]] + ['DSE'] + splt[dst_entity[0]:dst_entity[1]+1] + ['DEE'] + splt[dst_entity[1]+1:]
else:
  new_text=splt[0:dst_entity[0]] + ['DSE'] + splt[dst_entity[0]:dst_entity[1]+1] + ['DEE'] + splt[dst_entity[1]+1:src_entity[0]] + ['SSE'] + splt[src_entity[0]:src_entity[1]+1] + ['SEE'] + splt[src_entity[1]+1:]
print(new_text)
output=encoder.encode_plus(new_text, add_special_tokens=True, padding='max_length', max_length=25,truncation=True, return_tensors="pt")

print(output.tokens)
# train_df[0:5]


then terrorism struck again , this time in the indonesia capital of jakarta . [12, 12] [9, 9]
['then', 'terrorism', 'struck', 'again', ',', 'this', 'time', 'in', 'the', 'DSE', 'indonesia', 'DEE', 'capital', 'of', 'SSE', 'jakarta', 'SEE', '.']
<bound method BatchEncoding.tokens of {'input_ids': tensor([[  101,  1173, 12010,  4168,  1254,   117,  1142,  1159,  1107,  1103,
         28998,   100, 28999,  2364,  1104, 28996,   100, 28997,   119,   102,
             0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0]])}>


In [7]:
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
class nyt29dataset(Dataset):
  def __init__(self, df, encoder):
    self.comb_dat=df
    self.text=df['text']
    self.labels=df['labels']
    self.src_entity=df['source_entity']
    self.dst_entity=df['destination_entity']
    self.encoder=encoder
  
  def __len__(self):
    return self.labels.size

# will have to add special token so that the entity id is not lost
  def __getitem__(self,item):
    text=self.text[item]
    label=self.labels[item]
    src_ent=self.src_entity[item]
    dst_ent=self.dst_entity[item]
    splt=text.split(' ')
    if (src_ent[1]<dst_ent[0]):
      new_text=splt[0:src_ent[0]] + ['SSE'] + splt[src_ent[0]:src_ent[1]+1] + ['SEE'] + splt[src_ent[1]+1:dst_ent[0]] + ['DSE'] + splt[dst_ent[0]:dst_ent[1]+1] + ['DEE'] + splt[dst_ent[1]+1:]
    else:
      new_text=splt[0:dst_ent[0]] + ['DSE'] + splt[dst_ent[0]:dst_ent[1]+1] + ['DEE'] + splt[dst_ent[1]+1:src_ent[0]] + ['SSE'] + splt[src_ent[0]:src_ent[1]+1] + ['SEE'] + splt[src_ent[1]+1:]    
    encoding=encoder.encode_plus(new_text, add_special_tokens=True, padding='max_length', max_length=512,truncation=True, return_tensors="pt")
    sse=(encoding['input_ids'] == 28996).nonzero()[0][1]
    see=(encoding['input_ids'] == 28997).nonzero()[0][1]
    dse=(encoding['input_ids'] == 28998).nonzero()[0][1]
    dee=(encoding['input_ids'] == 28999).nonzero()[0][1]
    # a=np.full((512),-1)
    # b=np.full((512),-1)
    # for i,pos in enumerate(range(sse+1,see)):
    #   a[i]=pos
    # for i,pos in enumerate(range(dse+1,dee)):
    #   b[i]=pos
    # b=list(range(dse+1,dee))
    # print(sse,see,dse,dee,a,b)
    # print(encoding['input_ids'].shape,encoding['attention_mask'].shape,label.shape)
    # return encoding['input_ids'].squeeze(0), encoding['attention_mask'].squeeze(0),label,torch.tensor([sse+1,see]),torch.tensor([dse+1,dee])
    return encoding['input_ids'].squeeze(0), encoding['attention_mask'].squeeze(0),label,sse,see,dse,dee



In [8]:
train_dataset=nyt29dataset(train_df, encoder)
train_dataloader=DataLoader(train_dataset, batch_size=8)
val_dataset=nyt29dataset(val_df, encoder)
val_dataloader=DataLoader(val_dataset, batch_size=8)
inputids, attentionmasks, label, sse, see,dse,dee=next(iter(train_dataloader))
print(sse,see,dse,dee)

tensor([15, 10, 17, 17, 35, 35, 22, 20]) tensor([17, 12, 20, 20, 37, 37, 26, 24]) tensor([10, 15, 35, 22, 17, 20, 17, 35]) tensor([12, 17, 37, 26, 20, 24, 20, 37])


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
class ml_relation_classifier(nn.Module):
  def __init__(self):
    model = DistilBertModel.from_pretrained('distilbert-base-cased')
    model.resize_token_embeddings(len(encoder)) 
    super(ml_relation_classifier,self).__init__()
    self.DBert = model
    self.classifier = nn.Sequential(
          nn.Dropout(0.1),
          nn.Linear(3840, 30),
        )
  def batch_op_get_maxpooled_ent(se, ee, hidden_state):
    max_ent=torch.empty((8,768), dtype=torch.float)
    iter=0
    for i,j in zip(se+1,ee):
      a=list(range(i,j))
      # print(hidden_state[0][iter,a,:].shape)
      entity,indices=torch.max(hidden_state[0][iter,a,:],dim=0)
      max_ent[iter,:]=entity
      iter+=1
  def forward(self,input_ids,attention_masks,sse,see,dse,dee):
    # print(dst_ent_span)
    # for dst in range(dst_ent_span[:][0],dst_ent_span[:][1]):
    #   print(dst)
    # print(dst_ent=dst_ent_span if (dst_ent_span>=0))
    # print((dst_ent_span[:]>=0).nonzero(as_tuple=True))
    hidden_state=self.DBert(input_ids,attention_masks)
    # print(hidden_state[0][:,dst_ent_span,:].shape)
    # print(sse.shape)
    cls=hidden_state[0][:,0,:]
    sse=hidden_state[0][range(0,8),sse,:]
    see=hidden_state[0][range(0,8),see,:]
    dse=hidden_state[0][range(0,8),dse,:]
    dee=hidden_state[0][range(0,8),dee,:]
    # print(cls.shape,sse.shape)
    input=torch.cat((cls, sse, see,dse,dee), dim=1)
    # iter=0
    # for i,j in zip(sse+1,see):
    #   a=list(range(i,j))
    #   # print(hidden_state[0][iter,a,:].shape)
    #   entity,indices=torch.max(hidden_state[0][:,a,:],dim=0)
    #   mp_src_ent[iter,:]=entity
    #   iter+=1
    # iter=0
    # for i,j in zip(dse+1,dee):
    #   a=list(range(i,j))
    #   # print(hidden_state[0][iter,a,:].shape)
    #   entity,indices=torch.max(hidden_state[0][iter,a,:],dim=0)
    #   mp_src_ent[iter,:]=entity
    #   iter+=1
    # print(cls,mp_src_ent,mp_dst_ent)
    # print(cls.shape,mp_src_ent.shape,mp_dst_ent.shape)
    # mp_src_ent = mp_src_ent.to(device)
    # mp_dst_ent = mp_dst_ent.to(device)
    # print(cls.get_device(),mp_src_ent.get_device(),mp_dst_ent.get_device())
    # input=torch.cat((cls, mp_src_ent, mp_dst_ent), dim=1)
    # print('input')
    # print(input)
    # input.to(device)
    # print(input.shape)
    output=self.classifier(input)
    # print('output')
    # print(output)
    # logits=torch.sigmoid(output)
    return output
    # return logits
rel_ex=ml_relation_classifier()
rel_ex(inputids,attentionmasks,sse,see,dse,dee)

cuda:0


Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[ 6.7844e-01, -2.0489e-01,  9.2212e-02,  1.4837e-01, -1.6900e-01,
         -1.2007e-01,  3.9719e-01,  4.5350e-01,  7.5255e-02, -4.6530e-02,
         -2.3681e-02,  4.8042e-01, -2.4766e-01,  1.9262e-01,  5.0460e-01,
         -3.8832e-03, -9.4717e-02,  1.9432e-01, -2.7174e-01,  1.5057e-02,
         -4.2471e-02,  2.0808e-01, -5.7133e-01,  6.3727e-02,  2.2915e-01,
          3.2209e-01, -3.2095e-02,  2.5103e-01, -2.1716e-01,  2.4159e-01],
        [ 4.8893e-01, -2.7876e-01, -8.1300e-02,  1.1518e-01, -1.2056e-01,
         -8.5771e-02,  3.0293e-01,  4.9107e-01,  1.9307e-01, -1.1917e-01,
         -1.3764e-01,  4.0252e-01, -2.3123e-01,  1.9459e-01,  4.3851e-01,
         -2.9853e-02, -2.6557e-01,  8.9576e-02, -2.4030e-01,  1.1014e-01,
          4.0426e-02,  2.4999e-01, -4.3879e-01,  6.3840e-02,  1.8220e-01,
          3.8650e-01,  4.3892e-02,  1.8680e-01, -1.0683e-01,  1.5152e-01],
        [ 4.7288e-01,  3.7371e-02,  4.5432e-02, -2.2632e-01, -1.5324e-01,
         -3.2251e-01,  5.9737e-01,  

In [10]:
from transformers import AdamW
from transformers import get_scheduler
def init_model():
  classifier=ml_relation_classifier()
  classifier.to(device)
  optimizer = AdamW(classifier.parameters(), lr=5e-5)
  num_epochs = 1
  num_training_steps = num_epochs * len(train_dataloader)
  print(num_training_steps)
  lr_scheduler = get_scheduler(
      "linear",
      optimizer=optimizer,
      num_warmup_steps=0,
      num_training_steps=num_training_steps
  )
  threshold=0.4
  return classifier,optimizer,lr_scheduler,threshold,num_epochs

loss_f=nn.BCEWithLogitsLoss()



In [11]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
def compute_metrics(preds,labels):
  preds=preds.tolist()
  labels=labels.tolist()
  print(len(preds[3]),len(labels[5]))
  for pred in preds: del pred[-1] 
  for label in labels: del label[-1] 
  print(len(preds[3]),len(labels[5]))
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
  acc = accuracy_score(labels, preds)
  print("Precision = ", precision)
  print("recall = ", recall)
  print("F1 score = ", f1)
  print("accuracy = ", acc)

In [ ]:
import itertools
ml_rel_model,optimizer,lr_scheduler,threshold,num_epochs=init_model()
ml_rel_model.train()
for epoch in range(num_epochs):
  rev_preds=[]
  rev_labels=[]
  iter=0
  for input_ids,attention_masks,labels,sse, see,dse,dee in train_dataloader:
    iter+=1
    input_ids=input_ids.to(device)
    attention_masks=attention_masks.to(device)
    labels=labels.to(device)
    ml_rel_model.zero_grad()
    logits=ml_rel_model(input_ids,attention_masks,sse, see,dse,dee)
    # print(logits)
    # print(labels.float())
    loss=loss_f(logits,labels.float())
    if iter%99==0: 
      print(loss)
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
  print("Terminated Epoch : ",epoch)
  print("Saving model!!")
  torch.save(ml_rel_model,'/content/drive/MyDrive/nlp_assignments/assignment2/aassignment2/ml_rel_model_att_concat_spec_token.bin')
  print("Proceed for evaluation")
  ml_rel_model.eval()
  for input_ids,attention_masks,labels,sse, see,dse,dee in val_dataloader:
    input_ids=input_ids.to(device)
    attention_masks=attention_masks.to(device)
    labels=labels.to(device)
    with torch.no_grad():
      logits=ml_rel_model(input_ids,attention_masks,sse, see,dse,dee)
    logits=torch.sigmoid(logits)
    predictions=torch.where(logits>threshold,1.,0.)
    rev_preds.extend(predictions)
    rev_labels.extend(labels)
  compute_metrics(torch.stack(rev_preds).cpu(), torch.stack(rev_labels).cpu()) 





Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


18324
tensor(0.0796, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0785, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0634, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.1396, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0518, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0585, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0493, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0494, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0194, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0258, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0422, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0460, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0359, device='cuda:0', gr

In [ ]:
torch.save(ml_rel_model,'/content/drive/MyDrive/nlp_assignments/assignment2/aassignment2/ml_rel_model_att_concat_soec_tok2.bin')